In [ ]:
!pip install transformers
!pip install editdistance

import editdistance
import pandas as pd
import transformers
import matplotlib.pyplot as plt
import torch
import torch.cuda
import sklearn
import numpy as np
import random

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import accuracy_score,f1_score
from sklearn.metrics import accuracy_score
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn import preprocessing
from collections import defaultdict
from google.colab import drive
from transformers import AutoTokenizer, AutoModelForMaskedLM
from transformers import AutoModelForSequenceClassification

drive.mount('/content/drive')
dataset_mlt = pd.read_csv('/content/drive/MyDrive/Maltese_Unimorph_database_final.txt', sep="  ", header=None, error_bad_lines=False)
#dataset_mlt_2 = pd.read_csv('/content/drive/MyDrive/mlt')
dataset_ita = pd.read_csv('/content/drive/MyDrive/ita')

dataset_copy_mlt=dataset_mlt.copy()
dataset_copy_ita=dataset_ita.copy()

list_data_mlt = dataset_copy_mlt.values.tolist()
list_data_ita = dataset_copy_ita[0:int(len(list_data_mlt)/5)].values.tolist()


In [ ]:
split_data_mlt=[]
split_data_ita=[]

def split(corpus, spit_data):
  for sets in corpus:
    for x in sets:
      parts = x.split("\t")
      spit_data.append(parts)

split(list_data_mlt,split_data_mlt)
split(list_data_ita,split_data_ita)

In [ ]:
# removing duplicates
def remove_duplicates(data_array):
    unique_data_array = []
    second_element_set = set()
    for subarray in data_array:
        if subarray[1] not in second_element_set:
            unique_data_array.append(subarray)
            second_element_set.add(subarray[1])
    return unique_data_array

unique_data_mlt = remove_duplicates(split_data_mlt)
unique_data_ita = remove_duplicates(split_data_ita)

#getting the longest word for padding 
len_longest=0
for string in unique_data_mlt:
    x=string[1]
    if len(x)>len_longest:
      len_longest=len(x)
print(len_longest)
for string in unique_data_ita:
    x=string[1]
    if len(x)>len_longest:
      len_longest=len(x)


In [ ]:
# splitting into words and grammar
def split_word_and_grammar(unique_data_array):
    last_elements = [subarray[-1] for subarray in unique_data_array]
    word_list = [(subarray[0],subarray[1]) for subarray in unique_data_array]
    return word_list, last_elements

word_list_mlt, last_elements_mlt = split_word_and_grammar(unique_data_mlt)
word_list_ita, last_elements_ita = split_word_and_grammar(unique_data_ita)

word_list_concat_mlt = [('M ' + tup[0], tup[1]) for tup in word_list_mlt]
word_list_concat_ita = [('I ' + tup[0], tup[1]) for tup in word_list_ita]

print(word_list_concat_mlt[0:100])

# splitting grammar
def splitting_grammar(last_elements):
    list_of_lists = []
    for info in last_elements:
        parts = info.split(';')
        num_parts = [int(part) if part.isdigit() else part for part in parts]
        list_of_lists.append(num_parts)
    return list_of_lists

list_of_lists_mlt = splitting_grammar(last_elements_mlt)
list_of_lists_ita = splitting_grammar(last_elements_ita)
print(list_of_lists_mlt[0:100])

In [ ]:
# splitting data
# tokeniser = AutoTokenizer.from_pretrained("MLRS/mBERTu",char_level=True) #,char_level=True
# model = AutoModelForMaskedLM.from_pretrained("MLRS/mBERTu")

tokeniser = AutoTokenizer.from_pretrained("bert-base-multilingual-cased",char_level=True)
model = AutoModelForMaskedLM.from_pretrained("bert-base-multilingual-cased")

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

# Combine the word lists and grammar lists into one dataset
data_mlt = list(zip(word_list_concat_mlt, list_of_lists_mlt))
data_ita = list(zip(word_list_concat_ita, list_of_lists_ita))

# Split the Maltese data into test and train sets
test_size = int(len(data_mlt) * 0.1) # 10% of the Maltese data for testing
train_size = len(data_mlt) - test_size # remaining Maltese data for training

random.shuffle(data_mlt) # shuffle the Maltese data before splitting

# Split the Maltese data into train and test sets
test_data = data_mlt[:test_size]
train_data = data_mlt[test_size:] + data_ita

random.shuffle(train_data) # shuffle the training data before splitting

# Separate the x and y values for each dataset
X_train = [item[0] for item in train_data]
y_train = [item[1] for item in train_data]

X_test = [item[0] for item in test_data]
y_test = [item[1] for item in test_data]


In [ ]:
# tokenize the training set
token_indexes = tokeniser(X_train, return_tensors='pt', padding=True, truncation=True, is_split_into_words=False, max_length=512)
indexed_train_x = token_indexes['input_ids'].to(device).squeeze()
mask_train_x = token_indexes['attention_mask'].to(device).squeeze()

# tokenize the test set
new_tuples = [(("",) + t[1:]) for t in X_test]
token_indexes_test = tokeniser(new_tuples, return_tensors='pt', padding=True, truncation=True, is_split_into_words=False, max_length=512)
indexed_test_x = token_indexes_test['input_ids'].to(device).squeeze()
mask_test_x = token_indexes_test['attention_mask'].to(device).squeeze()

# padding w/0 so gramm info is all the same length
def padding_grammar(Y):
    max_len = max(len(info) for info in Y)
    padded_array = [info + ['<PAD>'] * (max_len - len(info)) for info in Y]
    return padded_array
padded_morph_info_train = padding_grammar(y_train)
padded_morph_info_test = padding_grammar(y_test)

# creating a dictionary for mapping 
morph_vocab = defaultdict(lambda: len(morph_vocab))
for i in range(len(padded_morph_info_train[0])):
    # extracting the unique values at this position
    values = set(info[i] for info in padded_morph_info_train)
    # assignign index
    for value in values:
        morph_vocab[value]
print(morph_vocab)

In [ ]:
# map the morphological information to indexes using the vocabulary
def get_targets_one_hot(padded_info):
  targets = [[morph_vocab[value] for value in info] for info in padded_info]
  targets = [[morph_vocab['<PAD>'] if value == 0 else value for value in info] for info in targets]
  targets=torch.tensor(targets).to(device)

  # one-hot encode the targets
  num_classes = len(morph_vocab)
  targets = torch.nn.functional.one_hot(targets, num_classes=num_classes).to(torch.float32)
  targets = torch.tensor(targets, requires_grad=True).to(device)
  return targets

targets_train = get_targets_one_hot(padded_morph_info_train)
targets_test = get_targets_one_hot(padded_morph_info_test)

In [ ]:
class Model(torch.nn.Module):
    def __init__(self, bert):
        super().__init__()
        self.bert = bert
        self.hidden_layer1 = torch.nn.Linear(768, 512)
        self.batchnorm1 = torch.nn.BatchNorm1d(512)
        self.drop1 = torch.nn.Dropout(p=0.3)

        self.hidden_layer2 = torch.nn.Linear(512, 256)
        self.batchnorm2 = torch.nn.BatchNorm1d(256)
        self.drop2 = torch.nn.Dropout(p=0.1)

        self.hidden_layer3 = torch.nn.Linear(256, 128)
        self.batchnorm3 = torch.nn.BatchNorm1d(128)
        self.drop3 = torch.nn.Dropout(p=0.2)

        self.output_layer = torch.nn.Linear(256, 10*len(morph_vocab))
    
    def forward(self, x, mask):
        with torch.cuda.amp.autocast():
            vecs = self.bert(x, attention_mask=mask, output_hidden_states=True).hidden_states[8][:, 0, :]

            hidden1 = self.hidden_layer1(vecs)
            hidden1 = self.batchnorm1(hidden1)
            hidden1 = torch.relu(hidden1)
            hidden1 = self.drop1(hidden1)

            hidden2 = self.hidden_layer2(hidden1)
            hidden2 = self.batchnorm2(hidden2)
            hidden2 = torch.relu(hidden2)
            hidden2 = self.drop2(hidden2)

            '''
            hidden3 = self.hidden_layer3(hidden2)
            hidden3 = self.batchnorm3(hidden3)
            hidden3 = torch.relu(hidden3)
            hidden3 = self.drop3(hidden3)
            '''

            output = self.output_layer(hidden2).view(-1, 10, len(morph_vocab))
            output = torch.sigmoid(output)
        return output


#bert = transformers.BertForMaskedLM.from_pretrained('MLRS/mBERTu')
bert = transformers.BertForMaskedLM.from_pretrained('bert-base-multilingual-cased')
model = Model(bert)
model.to(device)

In [ ]:
def grammatical_accuracy(output, target):
    output = output.detach().cpu().numpy()
    target = target.detach().cpu().numpy()

    output = np.argmax(output, axis=1)
    target = np.argmax(target, axis=1)
    accuracy = np.mean(output == target) * 100
    return accuracy

accuracy_count=[]
def get_acc(matches):
    if len(matches) == 0:
        return 0
    accuracy_count.append(sum(matches)/len(matches))
    return sum(matches)/len(matches)
#return round(100*sum(preds)/len(preds),3)

In [ ]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
batch_size=64

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5, weight_decay=0.1)
#optimizer = torch.optim.Adagrad(model.parameters(), lr=1e-4, weight_decay=0.01)
#scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=100, verbose=True)

print('step', 'error', 'accuracy')
train_errors = []
loss_fn = torch.nn.CrossEntropyLoss()
all_true=[]
all_pred=[]
accumulation_steps = 4
for step in range(1, 2000+1):
    optimizer.zero_grad()
    model.train(True)
    #.BCELoss()
    #.CrossEntropyLoss()
    #.BCEWithLogitsLoss()
   
    all_outputs = []
    all_targets = []
    for i in range(accumulation_steps):
    # print(targets_train.shape)
      output = model(indexed_train_x[i*batch_size:(i+1)*batch_size], mask_train_x[i*batch_size:(i+1)*batch_size])

      # MSELoss   
    #  output=output.to(torch.float16)
    #  targets_train=targets_train.to(torch.float16)

     # m = torch.nn.Sigmoid()
      error = loss_fn(output, targets_train[i*batch_size:(i+1)*batch_size])
      error = error/accumulation_steps
    #  scheduler.step(error)
      error.backward()

      all_outputs.append(output.cpu().detach().numpy())
      all_targets.append(targets_train[i*batch_size:(i+1)*batch_size].cpu().detach().numpy())

    optimizer.step()
    model.train(False)

    train_errors.append(error.detach().tolist())

    train_accs = []
    
  #  one_hot_output = np.round(output.cpu().detach()).numpy().astype(int)
    
    for x in range(0,batch_size): # every vector in the output
      targets = targets_train[i*batch_size:(i+1)*batch_size].cpu().detach()
      targets = targets[x].argmax(dim=1).tolist()
      predicted_labels = output.cpu().detach()[x].argmax(dim=1).tolist()
      results=0

      all_true.append(targets)
      all_pred.append(predicted_labels)
     # print('t',targets)
     # print('p',predicted_labels)
      for vecs1 in predicted_labels: # every vector in output vector i
        for vecs2 in targets: # every vector in target vector i
            if vecs1==vecs2:
                results=results+1
           
      if results==0.7*len(predicted_labels): #0.7*len(predicted_labels):
        train_accs.append(1)
      else:
        train_accs.append(0)  
    accuracy=get_acc(train_accs)


    if step % 10 == 0:
        print(step, train_errors[-1], accuracy)


In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import accuracy_score,f1_score
!pip install editdistance
import editdistance

m = MultiLabelBinarizer().fit(all_true)

f1_weighted=f1_score(m.transform(all_true),
         m.transform(all_pred),
         average='weighted')

f1_macro=f1_score(m.transform(all_true),
         m.transform(all_pred),
         average='macro')

print("F1 score weighted:", f1_weighted)
print("F1 score macro:", f1_macro)


all_true_flat = [label for sublist in all_true for label in sublist]
all_pred_flat = [label for sublist in all_pred for label in sublist]

f1_micro = f1_score(all_true_flat, all_pred_flat, average='micro')
print("F1 score micro:", f1_micro)

#problem with lev distance, uses order, order doesnt matter a lot here
reverse_dict = {v: k for k, v in morph_vocab.items()} 
distances = []
for i in range(len(all_true)):
    predicted_labels = [reverse_dict[j] for j in all_pred[i]]
    true_labels = [reverse_dict[j] for j in all_true[i]]
    distance = editdistance.eval(predicted_labels, true_labels)
    distances.append(distance)

average_distance = sum(distances) / len(distances)
print("Levenshtein distances:", average_distance)

avg= sum(accuracy_count)/len(accuracy_count)
print("the average is: ", avg)

In [ ]:
with torch.no_grad():
    print('sent', 'prediction')
    for i in range(len(indexed_test_x) // batch_size):
        outputs = model(indexed_test_x[i*batch_size:(i+1)*batch_size], mask_test_x[i*batch_size:(i+1)*batch_size])

        # reversing the dict to get the grammar from the index
        reverse_dict = {v: k for k, v in morph_vocab.items()} 

        for j, x in enumerate(outputs):
            predicted_labels = [reverse_dict[i] for i in x.argmax(dim=1).tolist()]

            # changing from char to word
            input_word = ''.join(tokeniser.decode(indexed_test_x[i*batch_size:(i+1)*batch_size][j]).split())

            print(input_word, predicted_labels)
            print("actual")
            print(y_test[i*batch_size:(i+1)*batch_size][j])

In [ ]:
(fig, ax) = plt.subplots(1, 1)
ax.set_xlabel('step')
ax.set_ylabel('$E$')
ax.plot(range(1, len(train_errors) + 1), train_errors, color='blue', linestyle='-', linewidth=3)
ax.grid()

print('   ')
(fig, ax) = plt.subplots(1, 1)
ax.set_xlabel('step')
ax.set_ylabel('$A$')
ax.plot(range(1, len(accuracy_count) + 1), accuracy_count, color='blue', linestyle='-', linewidth=3)
ax.grid()